### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [1]:
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [2]:
reviews.sort_values(by='timestamp').head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
498923,37287,2171847,6,1362062307,2013-02-28 14:38:27,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
442554,33140,444778,8,1362062624,2013-02-28 14:43:44,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
81920,6338,1411238,6,1362062838,2013-02-28 14:47:18,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
584570,43691,1496422,7,1362063503,2013-02-28 14:58:23,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
450669,33799,118799,5,1362063653,2013-02-28 15:00:53,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [3]:
def create_train_test(reviews, order_by='timestamp', 
                      training_size=8000, testing_size=2000):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    data = reviews.sort_values(by=order_by)
    training_df = data.iloc[:training_size, :]
    validation_df = data.iloc[training_size: training_size + testing_size, :]

    return training_df, validation_df

In [4]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [5]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [6]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        tic = time()

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])
        toc = time()

        # print results
        print("%d \t\t %f  time: %f" % (iteration+1, sse_accum / num_ratings, (toc - tic)))
        
    return user_mat, movie_mat 

In [7]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 10.395230  time: 4.081991
2 		 5.874445  time: 3.906617
3 		 4.117617  time: 3.865347
4 		 3.085255  time: 4.140859
5 		 2.402641  time: 3.970996
6 		 1.918926  time: 4.025306
7 		 1.560221  time: 3.829280
8 		 1.285795  time: 3.824778
9 		 1.071125  time: 3.956455
10 		 0.900393  time: 3.976985
11 		 0.762837  time: 3.822803
12 		 0.650814  time: 3.907154
13 		 0.558724  time: 4.108656
14 		 0.482366  time: 4.316761
15 		 0.418544  time: 4.152580
16 		 0.364801  time: 4.084175
17 		 0.319237  time: 4.350464
18 		 0.280369  time: 4.095386
19 		 0.247032  time: 3.904290
20 		 0.218300  time: 3.846229
21 		 0.193432  time: 3.844421
22 		 0.171827  time: 3.821881
23 		 0.152991  time: 3.908724
24 		 0.136521  time: 3.908894
25 		 0.122077  time: 3.883462
26 		 0.109380  time: 3.897055
27 		 0.098189  time: 3.865851
28 		 0.088306  time: 3.867147
29 		 0.079557  time: 3.867465
30 		 0.071798  time: 4.588709
31 		 0.064903  time:

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [10]:
train_data_df.head()

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
user_id = 51
train_data_df.index.get_loc(user_id)

3

In [13]:
movie_id = 14538
train_data_df.columns.get_loc(movie_id)

4

In [14]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''    
    # Get user and movie indexes
    user_idx = train_data_df.index.get_loc(user_id)
    movie_idx = train_data_df.columns.get_loc(movie_id)
    
    # User row and Movie Column
    user_row = user_matrix[user_idx, :]
    movie_col = movie_matrix[:, movie_idx]
    
    # Take dot product of that row and column in U and V to make prediction
    pred = user_row.dot(movie_col)
   
    return pred

In [15]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

6.351487125957071

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [28]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    movie_name = movies[movies.movie_id == movie_id].movie.values[0]
    print('The predicted rating for user {}, for movie ["{}"] is: {}'.format(
        user_id, movie_name, prediction))

In [29]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

The predicted rating for user 8, for movie ["Fantômas - À l'ombre de la guillotine (1913)"] is: 6.351487125957071


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well we have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [32]:
val_df.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
650588,49056,1598822,8,1363308721,2013-03-15 00:52:01,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650569,49056,289879,9,1363308742,2013-03-15 00:52:22,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650585,49056,1563738,9,1363308780,2013-03-15 00:53:00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650583,49056,1458175,4,1363308799,2013-03-15 00:53:19,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
378686,28599,103639,8,1363309112,2013-03-15 00:58:32,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [47]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    for idx in range(num_preds):
        user_id, movie_id, rating = val_df[['user_id', 'movie_id', 'rating']].iloc[idx].values.tolist()
        print('User id: {}.   Movie id: {}'.format(user_id, movie_id))
        predicted = predict_rating(user_mat, movie_mat, user_id, movie_id)
        print_prediction_summary(user_id, movie_id, predicted)
        print('The actual rating was: {}.  (Error: {}%)'.format(
            rating, 100 *(predicted - rating) / rating))
        print('-'*120)
    
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)

User id: 49056.   Movie id: 1598822
The predicted rating for user 49056, for movie ["New Year's Eve (2011)"] is: 7.211383481946897
The actual rating was: 8.  (Error: -9.857706475663786%)
------------------------------------------------------------------------------------------------------------------------
User id: 49056.   Movie id: 289879
The predicted rating for user 49056, for movie ["The Butterfly Effect (2004)"] is: 8.416536367608687
The actual rating was: 9.  (Error: -6.482929248792363%)
------------------------------------------------------------------------------------------------------------------------
User id: 49056.   Movie id: 1563738
The predicted rating for user 49056, for movie ["One Day (2011)"] is: 7.603810964417445
The actual rating was: 9.  (Error: -15.513211506472835%)
------------------------------------------------------------------------------------------------------------------------
User id: 49056.   Movie id: 1458175
The predicted rating for user 49056, for 

In [48]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

User id: 49056.   Movie id: 1598822
The predicted rating for user 49056, for movie ["New Year's Eve (2011)"] is: 7.211383481946897
The actual rating was: 8.  (Error: -9.857706475663786%)
------------------------------------------------------------------------------------------------------------------------
User id: 49056.   Movie id: 289879
The predicted rating for user 49056, for movie ["The Butterfly Effect (2004)"] is: 8.416536367608687
The actual rating was: 9.  (Error: -6.482929248792363%)
------------------------------------------------------------------------------------------------------------------------
User id: 49056.   Movie id: 1563738
The predicted rating for user 49056, for movie ["One Day (2011)"] is: 7.603810964417445
The actual rating was: 9.  (Error: -15.513211506472835%)
------------------------------------------------------------------------------------------------------------------------
User id: 49056.   Movie id: 1458175
The predicted rating for user 49056, for 

KeyError: 287978

I think that a movie that didn't exist in the training set appeared in the validation set.